# Homework 4

In [ ]:
# You may utilize other packages if you are more familiar with them.
import pandas as pd
import numpy as np
import math
import nltk
import string
import random
import torch
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Word2vec in multiple languages
For homework 4, we will be coding a word2vec model using an english and a small russian parallel text corpus. The goal is to understand how a very simple word embedding functions and how one might begin using them to investigate machine translation. 


### Load data
For this homework, we will be working with a subset of the multi-lingual text corpus from Helsinki-NLP/Tatoeba-Challenge https://github.com/Helsinki-NLP/Tatoeba-Challenge.
The subset is a partial copy of the bilingual Russian-English, English-Finnish, and Finnish-Russian parts of the corpus. 

In [ ]:
# Finnish and Russian parallel corpora
finrus = pd.read_csv('finrus_hw4.csv', index_col=0)
# English and Russian parallel corpora
engrus = pd.read_csv('engrus_hw4.csv', index_col=0)
# English and Finnish parallel corpora
engfin = pd.read_csv('engfin_hw4.csv', index_col=0)

### 0) First Look
Take a look at some of the data. Print out a few sentence pairs, show a histogram of sentence lengths for each of the languages, etc. 

In [ ]:
### type code here ###

### 1) Preprocessing
We will be building a skip-gram word2vec embedding using this corpus. Our first step is to preprocess the text. 
The end results of preprocessing can be slightly different, but here we want to at least do these 4 steps.

1) tokenize

2) lowercase

3) remove punctuation

4) remove stop words (this step will be done in part 3 of the homework, do not include in your 'preprocess_sentence' function)

Make a single function to preprocess all of our English, Finnish, and Russian text.

In [ ]:
def preprocess_sentence(sentence):
    ### type code here ###
    
    
    
    return preprocessed_sentence

Now preprocess all of our russian and english sentences.

In [ ]:
pp_eng2fin = [preprocess_sentence(sentence, english_stopwords) for sentence in engfin['english']]
pp_eng2rus = [preprocess_sentence(sentence, russian_stopwords) for sentence in engrus['english']]
pp_fin2eng = [preprocess_sentence(sentence, russian_stopwords) for sentence in engfin['finnish']]
pp_fin2rus = [preprocess_sentence(sentence, russian_stopwords) for sentence in finrus['finnish']]
pp_rus2eng = [preprocess_sentence(sentence, russian_stopwords) for sentence in engrus['russian']]
pp_rus2fin = [preprocess_sentence(sentence, russian_stopwords) for sentence in finrus['russian']]

### 2) One-hot-encode
The next step is to one-hot-encode our text corpora.
To do this, first begin by creating a vocabulary for each of our languages.
There can be many ways to do this, but the outcome should have the format of a dictionary where the key is the word and the value is an int unique to that language.
Also in this function, produce a separate dictionary, for token counts consisting of a dictionary with {key, count}. This will be used to remove stopwords and later for negative sampling.

Then, write a function using the vocabularies to one-hot-encode single words.

Hint: allow your function to take in 2 sets of preprocessed text so that it can create a vocabulary for all of the text from a given language. Example: english vocabulary should include words from eng-rus and eng-fin.

In [ ]:
from collections import Counter

def create_vocabulary(preprocessed_corpus):
    ### type code here ###
    
    
    
    
    
    return vocabulary, token_counts

vocabulary_english, token_counts_english = create_vocabulary(pp_eng2fin, pp_eng2rus)
print('english vocabulary length:',len(vocabulary_english))

vocabulary_finnish, token_counts_finnish = create_vocabulary(pp_fin2eng, pp_fin2rus)
print('english vocabulary length:',len(vocabulary_finnish))

vocabulary_russian, token_counts_russian = create_vocabulary(pp_rus2eng, pp_rus2fin)
print('russian vocabulary length:',len(vocabulary_russian))

In [ ]:
def onehotencode(word, vocabulary):
    ### type code here ###
    
    
    
    return 

### 3) Zipf's law
Provide 3 plots, one for each of our corpora, to indicate whether they follow Zipf's law.

Then, using your plots, make a judgement call on where to threshold each of your vocabularies where high-frequency words will be considered stopwords. Then, filter out all of the stopwords from the preprocessed sentences: pp_eng2fin, pp_eng2rus, pp_fin2eng, pp_fin2rus, pp_rus2eng, and pp_rus2fin.

In [ ]:
# plot word frequencies
### type code here ###

In [ ]:
# filter out stop words
### type code here ###

### 4) Generate training data
Our training data will consist of pairs of word and contexts (target, [context1, context2, ...]). 
The contexts for a given word are generated using the skip-gram model.
Here, generate these training data where x is a list of the target words.

The window size refers to the distance we look to generate the skip gram.
For the sentence "Apricots are the most delicious fruit." and a window size of 2, the skip-grams for the target word "delicious" would be [(delicious, the), (delicious, most), (delicious, fruit)]. In this example stop words are not removed, this is not the case in the homework.

Hint: a single target word can have multiple context words within a skipgram model.

Hint: for each language, make sure to pass in all sentences for that language. Example, there are 5000 english sentences in the engfin corpus and 5000 english sentences in the russian corpus, thus it should include 10000 sentences. 

In [ ]:
window_size = 2

def generate_training_data(corpus1, corpus2, vocabulary, window_size):
    ### type code here ###
    
    
    
    
    
            
    return

x_english, y_english = generate_training_data(pp_eng2fin, pp_eng2rus, vocabulary_english, window_size)
x_finnish, y_finnish = generate_training_data(pp_fin2eng, pp_fin2rus, vocabulary_finnish, window_size)
x_russian, y_russian = generate_training_data(pp_rus2eng, pp_rus2fin, vocabulary_russian, window_size)

### 5) Negative sampling
In order to train this model efficiently, we will be using negative sampling. Here, the model will be trained to discern positive, true pairs of (target, context) from negative, false pairs of (target, context). This binary classification makes the model much faster to train.

Here, we use a hyper parameter m, where for every positive pair of words we will include m negative pairs of words. 

Generate a set of true pairs consisting of one target word and one context word. Then, for each true pair, generate a corresponding set of m false pairs. These negative words are sampled from the original corpus, so should be sampled not uniformly from the vocabulary but from the distribution of tokens. 

These pairs need corresponding labels stored in a separate list with 0 for false pair and 1 for true pair.

Then, create a function that pulls batches from this data for training. 

Hint: shuffling your data will assist in learning. 

In [ ]:
m = 2

def generate_negative_samples(x, y, m, vocabulary):
    ### type code here ###
    
    
    
    
    return 

pairs_english, labels_english = generate_negative_samples(x_english, y_english, m, vocabulary_english)
pairs_finnish, labels_finnish = generate_negative_samples(x_finnish, y_finnish, m, vocabulary_finnish)
pairs_russian, labels_russian = generate_negative_samples(x_russian, y_russian, m, vocabulary_russian)

In [ ]:
def generate_batch(pairs, labels, batch_size):
    ### type code here ###
        
        
        
    return batch_pairs, batch_labels

### 6) Word2vec
Now, use pytorch and/or numpy to implement your own word2vec model. Here is the original paper for the model: https://arxiv.org/abs/1301.3781

Word2vec is a simple word embedding that incorporates 2 fully connected layers, one to embed words into a latent space and one to decode that embedding. Word2vec comes in a few different flavors, here we want to implement the version with negative sampling. 

Hint: the objective for negative sampling is to classify pairs of words as true pairs or false, negatively sampled, pairs. If your objective is a softmax, you're implementing the wrong type of word2vec.

Below are some parameters I recommend, but you may tune them to assist your model. 

In [ ]:
batch_size = 32
embed_size = 100
LR = 0.001
epochs = 500

In [ ]:
### type code here ###

### 7) Results
Show that you embedding worked. 
Vectorize each word in our corpus using your 3 word2vec models. 
Use PCA (feel free to use sklearn) to visualize each of our 3 word embeddings. Indicate some way of showing that the training of the model was successful. Besides the PCA plots, do at least 2 other things.

Suggestions:
1) Chose a few words from the corpus and print them out over the PCA plots to indicate how similar they are.

2) Use cosine similarity to compare a few chosen pairs of words from the corpora.

3) Try retraining the models with a subset of the data and compare its ability to embed words. 

4) Perform a clustering method on the words and investigate the quality of the clusters.

In [ ]:
from sklearn.decomposition import PCA
### type code here ###

### 8) Short Answer
In your own words (2-3 sentences), explain what the model learns. Make sure your explanation applies to all 3 of the word2vec models in this homework. 

Write your answer below this.



### 9) Connect our corpora
Develop a method to connect our corpora. Grading for this part of the homework will be lenient, but take this chance to be creative and do your best to show that you've connected the corpora. 

Suggestions:
1) Retrain a word2vec model that simultaneously embeds both english and russian words

2) Create a mapping between the two embeddings generated

3) Identify some idioms in one language within our embedded corpora and assess how they manifest in the embedding of another language.  

Feel free to try some different things out here. The end result should be a figure with a short caption. 

In [ ]:
### type code here ###